In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../data/df_segmentation.csv", index_col=[0])
df["date_mensuelle"] = pd.to_datetime(df["date_mensuelle"])
df['date_trimestrielle'] = df['date_mensuelle'].dt.year.astype(str) + '_' + df['date_mensuelle'].dt.quarter.astype(str)

# MOC C

In [3]:
resultats = df.groupby("Classes").agg(moyenne_TARGET=("TARGET", "mean")).to_dict()["moyenne_TARGET"]

In [4]:
resultats

{1: 0.24321912034511825,
 2: 0.1326317821973354,
 3: 0.08159497151836574,
 4: 0.0542754044301488,
 5: 0.03653685245805381,
 6: 0.02628501257236419,
 7: 0.02181450447847443}

In [5]:
def Boostrapping_Classes(classe):
    df_classe = df[df['Classes'] == classe]
    tx_defaut_liste = []

    for _ in range(1000):
        echantillon = df_classe["TARGET"].sample(n=len(df_classe), replace=True)
        tx_defaut_liste.append(echantillon.mean())
        
    tx_defaut_liste = np.array(tx_defaut_liste)

    decile9 = np.percentile(tx_defaut_liste, 90)

    MOC_C = decile9 - resultats[classe]
    
    return(classe, resultats[classe], decile9, MOC_C)

In [6]:
MOC_C = pd.DataFrame(columns = ["Classe", "LRA", "Décile9", "Moc_C"])

In [7]:
for i in range(1,8):
    classe, LRA, decile9, MOC_C_classe = Boostrapping_Classes(i)
    MOC_C.loc[len(MOC_C)] = [classe, LRA, decile9, MOC_C_classe]

In [8]:
MOC_C

,Classe,LRA,Décile9,Moc_C
0,1.0,0.243219,0.246794,0.003575
1,2.0,0.132632,0.134526,0.001894
2,3.0,0.081595,0.083225,0.001630
3,4.0,0.054275,0.055275,0.001000
4,5.0,0.036537,0.037462,0.000925
5,6.0,0.026285,0.027455,0.001170
6,7.0,0.021815,0.023982,0.002167


# MOC A
### Crise Covid

In [49]:
avant_covid = df[df['date_trimestrielle'] < '2020_2']

taux_defaut_avant_covid = avant_covid.groupby('Classes')['TARGET'].mean()
taux_defaut_pendant_covid = df.groupby('Classes')['TARGET'].mean()

impact_covid_par_classe = (taux_defaut_avant_covid/taux_defaut_pendant_covid)-1

In [50]:
impact_covid_par_classe

Classes
1    0.002053
2   -0.003810
3   -0.001246
4   -0.009041
5   -0.003066
6   -0.011353
7   -0.066636
Name: TARGET, dtype: float64

In [51]:
variance_taux_defaut_pendant_covid = df.groupby('Classes')['TARGET'].var()
variance_taux_defaut_pendant_covid

Classes
1    0.184073
2    0.115043
3    0.074939
4    0.051330
5    0.035202
6    0.025595
7    0.021342
Name: TARGET, dtype: float64

In [59]:
simulations = pd.DataFrame()

for classe in impact_covid_par_classe.index:
    moyenne_impact = impact_covid_par_classe[classe]
    variance_impact = variance_taux_defaut_pendant_covid[classe]

    variance_impact = variance_impact if variance_impact > 0 else 0.001

    tirages = np.random.normal(moyenne_impact, variance_impact, 1000)

    simulations[classe] = tirages
    
simulations

,1,2,3,4,5,6,7
0,-0.346145,-0.051989,0.114444,-0.026010,-0.056267,-0.044933,-0.081634
1,-0.178987,-0.164608,-0.058947,0.094995,-0.017860,-0.026594,-0.041274
2,-0.128608,0.002690,-0.013009,0.028531,0.004325,-0.009335,-0.067733
3,0.197386,-0.171325,0.074937,-0.048536,-0.004639,-0.040266,-0.072280
4,-0.426201,0.059713,-0.017533,-0.062809,0.070576,-0.020448,-0.047354
...,...,...,...,...,...,...,...
995,-0.003313,-0.176039,0.056152,-0.002449,-0.006753,-0.022806,-0.045307
996,-0.119887,-0.062121,-0.095822,0.029216,0.013937,-0.005151,-0.083355
997,0.254355,0.208906,-0.047767,-0.038243,-0.061123,-0.041613,-0.045726
998,0.239641,0.072327,-0.117108,-0.025546,-0.029324,0.003206,-0.032865


In [65]:
moyenne_empirique = simulations.mean()
percentile_90 = simulations.quantile(0.90)
MOC_A = percentile_90 - taux_defaut_pendant_covid
MOC_A

1   -0.005899
2    0.013013
3    0.010834
4    0.004667
5    0.005521
6   -0.005061
7   -0.060818
dtype: float64

# Conclusion

In [81]:
MOC = pd.DataFrame(columns = ["Classe", "LRA", "MOC_A", "MOC_C"])
MOC["LRA"] = taux_defaut_avant_covid.reset_index(drop=True)
MOC["MOC_A"] = MOC_A.reset_index(drop=True).apply(lambda x : 0 if x < 0 else x)
MOC["MOC_C"] = MOC_C["Moc_C"].reset_index(drop=True)
MOC["Classe"] = range(1,8)
MOC["LRA + MOC"] = MOC["LRA"] + MOC["MOC_A"] + MOC["MOC_C"]

In [82]:
MOC

,Classe,LRA,MOC_A,MOC_C,LRA + MOC
0,1,0.243718,0.000000,0.003575,0.247294
1,2,0.132126,0.013013,0.001894,0.147033
2,3,0.081493,0.010834,0.001630,0.093957
3,4,0.053785,0.004667,0.001000,0.059451
4,5,0.036425,0.005521,0.000925,0.042870
5,6,0.025987,0.000000,0.001170,0.027156
6,7,0.020361,0.000000,0.002167,0.022528
